In [18]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import json

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.utils.data import Dataset

from transformers import BertModel
from transformers import BertTokenizer
from transformers import BertPreTrainedModel
from transformers.models.bert.modeling_bert import BertEmbeddings, BertEncoder, BertPooler
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions

from typing import *

In [19]:
df = pd.read_csv('CoLA_train.csv')
df

,Unnamed: 0,label,context
0,0,1,One more pseudo generalization and I'm giving up.
1,1,1,One more pseudo generalization or I'm giving up.
2,2,1,"The more we study verbs, the crazier they get."
3,3,1,Day by day the facts are getting murkier.
4,4,1,I'll fix you a drink.
...,...,...,...
8545,8545,0,Poseidon appears to own a dragon
8546,8546,0,Digitize is my happiest memory
8547,8547,1,It is easy to slay the Gorgon.
8548,8548,1,I had the strangest feeling that I knew you.


In [20]:
import random

prompt_data = pd.DataFrame(columns= ['Prompt', 'label'], index= range(8550))
for i in range(len(df)):
    selector = random.randint(0, 1)
    if selector: 
        prompt_data['Prompt'][i] = 'Is the grammar of this sentence correct ?'
        if df['label'][i] == 1: prompt_data['label'][i] = 1
        else : prompt_data['label'][i] = 0
    else :
        prompt_data['Prompt'][i] = 'Is the grammar of this sentence incorrect ?'
        if df['label'][i] == 1: prompt_data['label'][i] = 0
        else : prompt_data['label'][i] = 1

prompt_data
    

,Prompt,label
0,Is the grammar of this sentence incorrect ?,0
1,Is the grammar of this sentence correct ?,1
2,Is the grammar of this sentence correct ?,1
3,Is the grammar of this sentence incorrect ?,0
4,Is the grammar of this sentence correct ?,1
...,...,...
8545,Is the grammar of this sentence correct ?,0
8546,Is the grammar of this sentence correct ?,0
8547,Is the grammar of this sentence incorrect ?,0
8548,Is the grammar of this sentence correct ?,1


In [21]:
df = df.drop(columns=['label', 'Unnamed: 0'])

In [22]:
CoLA_prompt = pd.concat([df, prompt_data], axis = 1)
CoLA_prompt.columns = ['context1', 'context2', 'label']
CoLA_prompt.to_csv('CoLA_Prompt.csv')

,context,Prompt,label
0,One more pseudo generalization and I'm giving up.,Is the grammar of this sentence incorrect ?,0
1,One more pseudo generalization or I'm giving up.,Is the grammar of this sentence correct ?,1
2,"The more we study verbs, the crazier they get.",Is the grammar of this sentence correct ?,1
3,Day by day the facts are getting murkier.,Is the grammar of this sentence incorrect ?,0
4,I'll fix you a drink.,Is the grammar of this sentence correct ?,1
...,...,...,...
8545,Poseidon appears to own a dragon,Is the grammar of this sentence correct ?,0
8546,Digitize is my happiest memory,Is the grammar of this sentence correct ?,0
8547,It is easy to slay the Gorgon.,Is the grammar of this sentence incorrect ?,0
8548,I had the strangest feeling that I knew you.,Is the grammar of this sentence correct ?,1
